## Streamlit Workbook

This is for figuring out the kinks in the Streamlit app

In [128]:
# cd OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction/notebooks/final_notebooks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
from random import randint
import  random
import os
#os.chdir('C:/Users/tmcro/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction')
from cmath import nan
from bs4 import BeautifulSoup
import streamlit as st
import pickle
from datetime import datetime
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
os.chdir('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')

In [129]:
def get_next_events(url):
    # Returns next 3 UFC events from UFC.COM

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('div', class_='c-card-event--result__info')
    event1_txt = soup.find('div', class_='c-card-event--result__info').text
    event1_url = event1.find('a')['href']
    event1_url = 'https://www.ufc.com' + event1_url
    event1_title = event1_txt.split('\n')[1]
    event1_time = event1_txt.split('/')[1]

    data = pd.DataFrame({'event_title': [event1_title], 'event_url': [event1_url], 'event_date': [event1_time]})

    event2 = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info')
    event2_txt = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').text
    event2_url = event2.find('a')['href']
    event2_url = 'https://www.ufc.com' + event2_url
    event2_title = event2_txt.split('\n')[1]
    event2_time = event2_txt.split('/')[1]


    data = data.append({'event_title': event2_title, 'event_url': event2_url, 'event_date': event2_time}, ignore_index=True)
    
    event3 = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info')
    event3_txt = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').text
    event3_url = event3.find('a')['href']
    event3_url = 'https://www.ufc.com' + event3_url
    event3_title = event3_txt.split('\n')[1]
    event3_time = event3_txt.split('/')[1]

    data = data.append({'event_title': event3_title, 'event_url': event3_url, 'event_date': event3_time}, ignore_index=True)
    
    return data

In [130]:
# Test
get_next_events('https://www.ufc.com/events')

,event_title,event_url,event_date
0,Poirier vs Gaethje 2,https://www.ufc.com/event/ufc-291,10:00 PM EDT
1,Sandhagen vs Font,https://www.ufc.com/event/ufc-fight-night-augu...,10:00 PM EDT
2,Luque vs Dos Anjos,https://www.ufc.com/event/ufc-fight-night-augu...,7:00 PM EDT


In [131]:
# Function to get the fight card for a given event
def get_event_fights(event_url):
    page = requests.get(event_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get main card, fight 1

    mcn = soup.find_all('li', class_='l-listing__item')
    # get num of mc
    num_mc = len(mcn)
    # for each mc, do the following
    data = pd.DataFrame()
    n = 0
    for i in mcn:
        mc = mcn[n]
        # fight 1
        fighter1= mc.find('div', class_ ='c-listing-fight__corner-name c-listing-fight__corner-name--red').text
        fighter1 = fighter1.replace('\n', ' ')
        fighter1 = fighter1.strip()
        fighter2 = mc.find('div', class_ ='c-listing-fight__corner-name c-listing-fight__corner-name--blue').text
        fighter2 = fighter2.replace('\n', ' ')
        fighter2 = fighter2.strip()
        weightclass = mc.find('div', class_='c-listing-fight__class-text').text
        fighter1_odds = mc.find('span', class_='c-listing-fight__odds').text
        fighter2_odds = mc.find('span', class_='c-listing-fight__odds').find_next('span', class_='c-listing-fight__odds').text
        fighter1_odds = fighter1_odds.replace('\n', '')
        fighter2_odds = fighter2_odds.replace('\n', '')
        # fighter odds to float
        if (fighter1_odds == '-') :
            fighter1_odds = nan
        if (fighter2_odds == '-') :
            fighter2_odds = nan

        data = data.append({'fighter1': fighter1, 'fighter2': fighter2, 'weightclass': weightclass, 
                            'fighter1_odds': fighter1_odds, 'fighter2_odds': fighter2_odds}, ignore_index=True)
        n = n + 1
    return data

In [132]:
def secret_number(event_url):
    # if no driver open, open one
    driver = None
    if (driver == None):
        driver = webdriver.Chrome('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2/chromedriver')
    else:
        driver = driver
    
    driver.get(event_url)
    time.sleep(3)
    # click the first matchup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pretty = soup.prettify()
    # find first data-fmid to get first matchup
    fmid_start = pretty.find('data-fmid')
    fmid = pretty[fmid_start+11:fmid_start+16]
    driver.get(event_url +'#' + fmid)
    time.sleep(6)
    # find all links within page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # find all iframe src
    iframe = soup.find_all('iframe')
    # find all links
    iframe_text = str(iframe)
    matchup = iframe_text.find('matchup')
    matchup_url = iframe_text[matchup+8:matchup+12]
    print('matchup_url: ' + matchup_url)
    secret_number = matchup_url
    return secret_number

In [133]:
def get_next_events2(url):
    # get next events if event fighter data is not na

    data = get_next_events(url)
    for i in range(0, len(data)):
        event_url = data['event_url'][i]
        event_fights = get_event_fights(event_url)
        if (len(event_fights) == 0):
            data = data.drop(i)
    return data

In [134]:
def get_next_event_ufcstats():
    url = 'http://www.ufcstats.com/statistics/events/upcoming'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('td', class_='b-statistics__table-col')
    event1_txt = soup.find('td', class_='b-statistics__table-col').text
    event_txt = event1_txt.replace('   ', '').replace('\n', '').strip()
    event_title = event_txt.split('  ')[0]
    event_date = event_txt.split('  ')[1]
    event1_url = event1.find('a')['href']
    data = pd.DataFrame({'event_title': [event_title], 'event_url': [event1_url], 'event_date': [event_date]})
    return data

In [135]:
def get_next_event_ufcstats():
    url = 'http://www.ufcstats.com/statistics/events/upcoming'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('td', class_='b-statistics__table-col')
    event1_txt = soup.find('td', class_='b-statistics__table-col').text
    event_txt = event1_txt.replace('   ', '').replace('\n', '').strip()
    event_title = event_txt.split('  ')[0]
    event_date = event_txt.split('  ')[1]
    event1_url = event1.find('a')['href']
    data = pd.DataFrame({'event_title': [event_title], 'event_url': [event1_url], 'event_date': [event_date]})
    return data

In [136]:
def get_fighter_urls(event_details_url):
    # Request the webpage
    page = requests.get(event_details_url) 

    # Parse the HTML content
    soup = BeautifulSoup(page.content, 'html.parser') 

    # Find all event rows
    events = soup.find_all('tr', class_='b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click') 

    n = 0
    next_event_data = pd.DataFrame()

    # Loop through each event
    for event in events:
        # Find all fighters' details
        fighters = events[n].find_all('p', class_='b-fight-details__table-text') 

        # Extract fighter names
        fighter1 = fighters[0].text.replace('  ', '').replace('\n', '').strip()
        fighter2 = fighters[1].text.replace('  ', '').replace('\n', '').strip()

        # Extract fighter URLs
        fighter1_url = fighters[0].find('a')['href']
        fighter2_url = fighters[1].find('a')['href']

        # Append the fighter details to the dataframe
        next_event_data = next_event_data.append({'fighter1': fighter1, 'fighter2:': fighter2, 'fighter1_url': fighter1_url, 'fighter2_url': fighter2_url, 'fight#': n + 1}, ignore_index=True)

        n += 1

    return next_event_data

In [137]:
def is_saturday():
    # check if it is a saturday
    today = str(datetime.today().weekday())
    if today == '5':
        return True
    else:
        return False

# if it is a saturday, get the current event, otherwise, get the next event


In [138]:
next_eventz = get_next_event_ufcstats()
next_eventz

,event_title,event_url,event_date
0,UFC 291: Poirier vs. Gaethje 2,http://www.ufcstats.com/event-details/ccd58ff7...,"July 29, 2023"


In [139]:
# make sure events have fight info. If not, disregard that event
next = get_next_events2('https://www.ufc.com/events')
next

,event_title,event_url,event_date
0,Poirier vs Gaethje 2,https://www.ufc.com/event/ufc-291,10:00 PM EDT
1,Sandhagen vs Font,https://www.ufc.com/event/ufc-fight-night-augu...,10:00 PM EDT
2,Luque vs Dos Anjos,https://www.ufc.com/event/ufc-fight-night-augu...,7:00 PM EDT


In [140]:
########           Select & Show Next Event    ################

event = next['event_title'][0]
selected_event = event
event_url =  next['event_url'][next['event_title'] == selected_event].values[0]
selected_event_secret_number = secret_number(event_url)

next_event = get_event_fights(event_url)
#fight = st.sidebar.selectbox('Select Fight', next_event['fighter1'] + ' vs. ' + next_event['fighter2'])

## Get Names ##

#selected_fighter_1 = fight.split(' vs. ')[0]
#selected_fighter_2 = fight.split(' vs. ')[1].strip()


########          Scrape UFC.com Data    ################

# get the matchup fight numbers

page = requests.get(event_url)
soup = BeautifulSoup(page.content, 'html.parser')
h = soup.find_all('div', class_='c-listing-fight')
data_fmid = []
for i in h:
    data_fmid.append(i['data-fmid'])

next_event['fight_number'] = data_fmid[:len(next_event)]
next_event['matchup_url'] = event_url +'#' + next_event['fight_number'].astype(str)
selected_matchup_url = next_event['matchup_url'][next_event['fighter1'] == selected_fighter_1].values[0]

#st.write(' FOR EDIT -- selected_matchup_url: ' + selected_matchup_url)
next_event

matchup_url: 1155


,fighter1,fighter2,weightclass,fighter1_odds,fighter2_odds,fight_number,matchup_url
0,Dustin Poirier,Justin Gaethje,Lightweight Bout,−145,+125,10590,https://www.ufc.com/event/ufc-291#10590
1,Jan Błachowicz,Alex Pereira,Light Heavyweight Bout,−120,−110,10591,https://www.ufc.com/event/ufc-291#10591
2,Stephen Thompson,Michel Pereira,Welterweight Bout,−165,+140,10594,https://www.ufc.com/event/ufc-291#10594
3,Tony Ferguson,Bobby Green,Lightweight Bout,+285,−375,10592,https://www.ufc.com/event/ufc-291#10592
4,Michael Chiesa,Kevin Holland,Welterweight Bout,+125,−145,10596,https://www.ufc.com/event/ufc-291#10596
5,Gabriel Bonfim,Trevin Giles,Welterweight Bout,+260,−315,10662,https://www.ufc.com/event/ufc-291#10662
6,Derrick Lewis,Marcos Rogerio de Lima,Heavyweight Bout,+155,−180,10595,https://www.ufc.com/event/ufc-291#10595
7,Roman Kopylov,Claudio Ribeiro,Middleweight Bout,−220,+180,10674,https://www.ufc.com/event/ufc-291#10674
8,Jake Matthews,Darrius Flowers,Welterweight Bout,NaN,NaN,10751,https://www.ufc.com/event/ufc-291#10751
9,CJ Vergara,Vinicius Salvador,Flyweight Bout,−160,+130,10675,https://www.ufc.com/event/ufc-291#10675


In [163]:
# test url for UFC 291:
test_matchup_url = 'https://www.ufc.com/event/ufc-291#10590'

In [180]:
def get_secret_number(url):
    driver = webdriver.Chrome('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2/chromedriver')
    driver.get(url)
    time.sleep(4)
    # scrape data
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # find class = c-listing-fight__details-content details-content
    fight = soup.find('div', class_='c-listing-fight__details-content details-content')

    content_details = fight.find('div', class_='details-content__iframe-wrapper')
    # get the src from content details
    src = content_details.find('iframe')['src']
    # drop the /pre
    src = src.replace('/pre', '')
    secret_number = src.split('/')[2]
    return secret_number

In [181]:
secret_number = get_secret_number(test_matchup_url)
secret_number

'1155'

Add secret number to URLS to get Data URL

Final URL Should look like: UFC.com/matchup/secret_number/fight_number

In [191]:
# add secret number to next_event
next_event['secret_number'] = secret_number

In [192]:
next_event['fight_data_url'] = 'https://www.ufc.com/matchup/' + next_event['secret_number'] + '/' + next_event['fight_number'].astype(str)
next_event

,fighter1,fighter2,weightclass,fighter1_odds,fighter2_odds,fight_number,matchup_url,secret_number,fight_data_url
0,Dustin Poirier,Justin Gaethje,Lightweight Bout,−145,+125,10590,https://www.ufc.com/event/ufc-291#10590,1155,https://www.ufc.com/matchup/1155/10590
1,Jan Błachowicz,Alex Pereira,Light Heavyweight Bout,−120,−110,10591,https://www.ufc.com/event/ufc-291#10591,1155,https://www.ufc.com/matchup/1155/10591
2,Stephen Thompson,Michel Pereira,Welterweight Bout,−165,+140,10594,https://www.ufc.com/event/ufc-291#10594,1155,https://www.ufc.com/matchup/1155/10594
3,Tony Ferguson,Bobby Green,Lightweight Bout,+285,−375,10592,https://www.ufc.com/event/ufc-291#10592,1155,https://www.ufc.com/matchup/1155/10592
4,Michael Chiesa,Kevin Holland,Welterweight Bout,+125,−145,10596,https://www.ufc.com/event/ufc-291#10596,1155,https://www.ufc.com/matchup/1155/10596
5,Gabriel Bonfim,Trevin Giles,Welterweight Bout,+260,−315,10662,https://www.ufc.com/event/ufc-291#10662,1155,https://www.ufc.com/matchup/1155/10662
6,Derrick Lewis,Marcos Rogerio de Lima,Heavyweight Bout,+155,−180,10595,https://www.ufc.com/event/ufc-291#10595,1155,https://www.ufc.com/matchup/1155/10595
7,Roman Kopylov,Claudio Ribeiro,Middleweight Bout,−220,+180,10674,https://www.ufc.com/event/ufc-291#10674,1155,https://www.ufc.com/matchup/1155/10674
8,Jake Matthews,Darrius Flowers,Welterweight Bout,NaN,NaN,10751,https://www.ufc.com/event/ufc-291#10751,1155,https://www.ufc.com/matchup/1155/10751
9,CJ Vergara,Vinicius Salvador,Flyweight Bout,−160,+130,10675,https://www.ufc.com/event/ufc-291#10675,1155,https://www.ufc.com/matchup/1155/10675


In [193]:
def get_matchup_data(matchup_data_url):
    # gets the matchup data from the matchup data URL
    options = Options()
    options.headless = True
    driver = webdriver.Chrome('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2/chromedriver')
    driver.get(matchup_data_url)
    # print all data
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # get all text
    text = soup.get_text()
    # drop all \n\n
    text = text.replace('\n\n', '')
    # split on \n
    text = text.split('\n')
    # drop extra spaces
    text = [x.strip() for x in text]
    # drop all text before Matchup Stats
    text = text[text.index('Odds')+1:]
    # split on multiple spaces
    text = [x.split('  ') for x in text]
    # drop all empty lists
    text = [x for x in text if x != ['']]
    # drop all empty strings
    text = [[x for x in y if x != ''] for y in text]
    # split on ,
    text = [[x.split(',') for x in y] for y in text]
    # join to ONE list
    text = [item for sublist in text for item in sublist]
    # drop 'Per 15 Min' 
    text = [x for x in text if x != ['Per 15 Min']]
    # every 3 items is a new row
    text = [text[i:i+3] for i in range(0, len(text), 3)]
    # make into dataframe, column 1 is red_corner, column 2 is stat, column 3 is blue_corner
    df = pd.DataFrame(text)
    # name columns
    df.columns = ['red_corner', 'stat', 'blue_corner']
    # get rid of [] around values
    df['red_corner'] = df['red_corner'].str[0]
    df['stat'] = df['stat'].str[0]
    df['blue_corner'] = df['blue_corner'].str[0]
    # make red_corner row 0, blue corner row 1, and stat the column names
    df = df.T
    # make the stat row the column names
    df.columns = df.iloc[1]
    # drop the stat row
    df = df.drop(df.index[1])

    return df


In [194]:
test_source_2 = next_event['fight_data_url'][2]
test_source_2

'https://www.ufc.com/matchup/1155/10594'

In [195]:
get_matchup_data(test_source_2)

stat,Record,Last Fight,Country,Height,Weight,Reach,Leg Reach,KO/TKO,Sub,Dec,...,Defense,Takedown Avg,Takedown Accuracy,Takedown Defense,Submission Avg,Money Line,Total Rounds,Odds to Win by KO,Odds to Win by Submission,Odds to Win by Decision
red_corner,17-6-1,Win,United States,"6' 0""",170.00 LB,75 in,42 in,47%,6%,47%,...,55%,0.27,41%,64%,0,-170,N/A,N/A,N/A,N/A
blue_corner,28-11-0,Win,Brazil,"6' 1""",170.00 LB,73 in,42 in,37%,26%,37%,...,58%,1.58,55%,94%,0.57,+145,N/A,N/A,N/A,N/A
